In [62]:
import math
import random
from collections import defaultdict
import json
from sklearn import linear_model
import sklearn
import csv
import gzip
import string
import copy
import scipy
from implicit import bpr

In [63]:
#Helpful functions
def readGz(path):
    for l in gzip.open(path, 'rb'):
        yield eval(l)

In [64]:
#Exploring the data
reviewSmall = [] #Users and a list of games they have reviewed; Size: 25799
userToItem = [] #Users and a list of games they have downloaded; Size: 50000
#User id's ^ directly correlated 

reviewBig = [] #Random assortment of reviews; Size: 50000
itemMetaData = [] #Data about each individual game; Size: 15,474
bundleData = [] #Data about each bundle; Size: 615

i = 0
for d in readGz('australian_user_reviews.json.gz'):
    if i < 50000:
        reviewSmall.append(d)
    else:
        break
    i+=1

i = 0
for d in readGz('australian_users_items.json.gz'):
    if i < 50000:
        userToItem.append(d)
    else:
        break
    i+=1

i = 0
for d in readGz('bundle_data.json.gz'):
    if i < 50000:
        bundleData.append(d)
    else:
        break
    i+=1

i = 0
for d in readGz('steam_games.json.gz'):
    
    if i < 50000:
        itemMetaData.append(d)
    else:
        break
    i+=1

i = 0
for d in readGz('steam_reviews.json.gz'):
    if i < 50000:
        reviewBig.append(d)
    else:
        break
    i+=1

In [83]:
reviewSmall[3]

{'user_id': 'doctr',
 'user_url': 'http://steamcommunity.com/id/doctr',
 'reviews': [{'funny': '',
   'posted': 'Posted October 14, 2013.',
   'last_edited': '',
   'item_id': '250320',
   'helpful': '2 of 2 people (100%) found this review helpful',
   'recommend': True,
   'review': 'This game... is so fun. The fight sequences have been improved from walking dead. It also includes more of a Sam and Max puzzle solving (some of it in the first episode) and walking dead. The game also gets even more better if you have read the Fables comic books, which are without a doubt, very good. The music is also superb and fit the scenarios very well.'},
  {'funny': '',
   'posted': 'Posted July 28, 2012.',
   'last_edited': '',
   'item_id': '20920',
   'helpful': '1 of 1 people (100%) found this review helpful',
   'recommend': True,
   'review': "Really Really Really Great Game, very good story, im in chapter 1 atm and i think its great. You get a really early link with characters. No need to pl

In [88]:
userToItem[2]

{'user_id': 'evcentric',
 'items_count': 137,
 'steam_id': '76561198007712555',
 'user_url': 'http://steamcommunity.com/id/evcentric',
 'items': [{'item_id': '466170',
   'item_name': 'Idling to Rule the Gods',
   'playtime_forever': 28545,
   'playtime_2weeks': 1554},
  {'item_id': '224500',
   'item_name': 'Gnomoria',
   'playtime_forever': 13618,
   'playtime_2weeks': 0},
  {'item_id': '22380',
   'item_name': 'Fallout: New Vegas',
   'playtime_forever': 11311,
   'playtime_2weeks': 0}]}

In [15]:
reviewBig[1]

{'username': '₮ʜᴇ Wᴀʀᴛᴏɴ',
 'hours': 51.1,
 'products': 769,
 'product_id': '328100',
 'page_order': 0,
 'date': '2017-12-27',
 'text': 'looks like a facebook game',
 'early_access': False,
 'page': 1}

In [16]:
itemMetaData[0]

{'publisher': 'Kotoshiro',
 'genres': ['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'],
 'app_name': 'Lost Summoner Kitty',
 'title': 'Lost Summoner Kitty',
 'url': 'http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/',
 'release_date': '2018-01-04',
 'tags': ['Strategy', 'Action', 'Indie', 'Casual', 'Simulation'],
 'discount_price': 4.49,
 'reviews_url': 'http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1',
 'specs': ['Single-player'],
 'price': 4.99,
 'early_access': False,
 'id': '761140',
 'developer': 'Kotoshiro'}

In [95]:
bundleData[100]

{'bundle_final_price': '$21.23',
 'bundle_url': 'http://store.steampowered.com/bundle/1100/?utm_source=SteamDB&utm_medium=SteamDB&utm_campaign=SteamDB%20Bundles%20Page',
 'bundle_price': '$24.98',
 'bundle_name': 'New World Collection',
 'bundle_id': '1100',
 'items': [{'genre': 'Action, Indie, Strategy',
   'item_id': '222880',
   'discounted_price': '$4.99',
   'item_url': 'http://store.steampowered.com/app/222880',
   'item_name': 'Insurgency'},
  {'genre': 'Action, Indie, Strategy, Early Access',
   'item_id': '447820',
   'discounted_price': '$19.99',
   'item_url': 'http://store.steampowered.com/app/447820',
   'item_name': 'Day of Infamy'}],
 'bundle_discount': '15%'}

In [ ]:
#Predict whether user will buy a bundle based on user features, item features, bundle features (ex. discount), similarity scores? 
#^^^ All in a feature vector ^^^

#Our first task should be how to find out how to check if a user has bought a bundle; maybe assume that if user has bought
#every game in bundle, then user has bought that bundle

In [79]:
userTop3 = [d for d in userToItem] #Every user's top 3 items
for d in userTop3:
    items = [i for i in d['items']]
    items.sort(key=lambda x: x['playtime_forever'], reverse = True)
    d['items'] = items[:3]

In [96]:
userTop3[25000]

{'user_id': 'DefMind',
 'items_count': 44,
 'steam_id': '76561198131540855',
 'user_url': 'http://steamcommunity.com/id/DefMind',
 'items': [{'item_id': '304930',
   'item_name': 'Unturned',
   'playtime_forever': 2018,
   'playtime_2weeks': 73},
  {'item_id': '243870',
   'item_name': "Tom Clancy's Ghost Recon Phantoms - NA",
   'playtime_forever': 574,
   'playtime_2weeks': 0},
  {'item_id': '4000',
   'item_name': "Garry's Mod",
   'playtime_forever': 481,
   'playtime_2weeks': 481}]}